# Beispielaufgabe

### Fabrik hat 7 Produkte für welche folgende Maschinen Verwendet werden:

- 4 Schleifer
- 2 Vertikale Bohrer
- 3 horizontale Bohrer
- 1 Fräse
- 1 Hobel

**Gewinn: Preis - Materialkosten**

**Maschinen: Arbeitszeit (in h)**

|            | Prod1 | Prod2 | Prod3 | Prod4 | Prod5 | Prod7 | Prod7
|-----------:|:-----|:------|:------|:------|:------|:------|:------|
Gewinn | 10 | 6 | 8 | 4 | 11 | 9 | 3 |
Schleifen | 0.5 | 0.7 | - | - | 0.3 | 0.2 | 0.5 |
vertikal bohren | 0.1 | 0.2 | - | 0.3 | - | 0.6 | - |
horizontal bohren | 0.2 | - | 0.8 | - | - | - | 0.6 |
Fräsen | 0.05 | 0.03 | - | 0.07 | 0.1 | - | 0.08 |
hobeln | - | - | 0.01 | - | 0.05 | - | 0.05 |

### Aber: Maschinen müssen gewartet werden!

| Monat | Maschine |
|------:|---------:|
| Jan | 1 Schleifer |
| Feb | 2 h. Bohrer|
| Mär | 1 Fräse |
| Apr | 1 v. Bohrer|
| Mai | 1 Schleifer + 1 v. Bohrer |
| Jun | 1 h. Bohrer |


### Limits für Verkauf pro Monat:

| Monat | Prod1 | Prod2 | Prod3 | Prod4 | Prod5 | Prod7 | Prod7
|-----------:|:-----|:------|:------|:------|:------|:------|:------|
Jan | 500 | 1000 | 300 | 300 | 800 | 200 | 100 |
Feb | 600 | 500 | 200 | 0 | 400 | 300 | 150  |
Mär | 300 | 600 | 0 | 0 | 500 | 400 | 100 |
Apr | 200 | 300 | 400 | 500 | 200 | 0 | 100 |
Mai | 0| 100 | 500 | 100 | 1000 | 300 | 0 |
Jun | 500 | 500 | 100 | 300 | 1100 | 500 | 60 |


### Lagerung: 

- 100 Einheiten pro Produkt
- Kosten: 0.50 €
- Januar: Lager leer
- Juni: 50 Einheiten

### Arbeit: 
    
- 6 Tage / Woche, 2 * 8h Schicht
- 24 Arbeitstage / Monat

## Wie soll der Produktionsplan aussehen?

# Mathematische modellierung

### Sets

$T$ = Zeitdauer(Monate) -> $T_0$ = 1. Monat, $T_e$ = letzter 

$P$ = Produkte

$M$ = Maschinen


### Parameter

- Für jedes Produkt $p \in P$ und Maschine $m \in M$ ist die Zeit $f_{p,m}$ gegeben, in der das Produkt in der Maschine bearbeitet wird
- Für jeden Monat $t \in T$ und jedes Produkt $p \in P$ gibt es ein Limit $l_{t,p}$, wie oft es Verkauft werden kann
- Für jedes Produkt $p \in P$ gibt es einen Profit $k_p$
- Für jeden Monat $t \in T$ und jede Maschine $m \in M$ gibt es eine Anzahl an verfügbaren Maschinen $q_{t,m}$
- Jede Maschine arbeitet $h$ Stunden im Monat
- Es können $z$ Einheiten von jedem Produkt für die Kosten $r$ pro Monat gelagert werden 



### Variablen

Für jeden Monat $t$ und Produkt $p$ werden folgende nichtnegative Variablen eingeführt: $b_{t,p} , u_{t,p} , s_{t,p}$.

- $b_{t,p}$ beschreibt die Produktionsanzahl eines Produkts pro Monat
- $u_{t,p}$ beschreibt die Verkaufsanzahl eines Produktes pro Monat
- $s_{t,p}$ beschreibt die Lageranzahl eines Produktes pro Monat



### Objective funktion

**Objective/ Ziel: Maximaler Profit**

$max \sum_\limits{t\in T} \sum_\limits{p\in P} (k_p * u_{t,p} - r * s_{t,p})$

### Constraints


$s_{t-q,p} + b_{t,p} = u_{t,p} + s_{t,p} \forall t \in T \ t_0, \forall p \in P$

$b_{t_0,p} = u_{t_0,p} + s_{t_0,p} \forall p \in P$

"Balance Constraints": Es können nur so viele Produkte gekauft($u_{t,p}$) und eingelagert($s_{t,p}$) werden, wie im Vorherigen Monat gelagert($s_{t-q,p}$) waren und in diesem Monat produziert($b_{t,p}$) wurden.


$s_{t_e,p} = z \forall p \in P$

"Endstore Constraints": Am Ende des letzten Monats muss das Lageranzahl($s_{t_e,p}$) für jedes Produkt dem maximalen Lagerplatz($z$) entsprechen  


$s_{t,p} \leq z \forall p \in P, \forall t \in T$

"Store Capacity Constraints": Es es können nich mehr Produkte eingelagert werden($s_{t,p}$), als es Lagerplatz($z$) gibt.


$ \sum_\limits{p\in P} f_{p,m} * b_{t,p} \leq h * q_{t,m} \forall t \in T, \forall m \in M$

"Capacity Constraints": Die Zeit in der alle Produkte an den jeweiligen Maschinen produziert werden muss kleiner sein als die Gesamtarbeitszeit($h$) aller verfügbaren Maschinen($q_{t,m}$)

# code!!

In [1]:
from gurobipy import *

In [2]:
products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]
machines = ["Schleifer", "vBohrer", "hBohrer", "Fräse","Hobel"]

In [3]:
profit = {"Prod1": 10,
          "Prod2": 6,
          "Prod3": 8,
          "Prod4": 4,
          "Prod5": 11,
          "Prod6": 9,
          "Prod7": 3,}

In [4]:
qMachine = {"Schleifer": 4,
            "vBohrer": 2,
            "hBohrer": 3,
            "Fräse": 1,
            "Hobel": 1,}

In [5]:
time_table = {"Schleifer":{  "Prod1": 0.5,
                             "Prod2": 0.7,
                             "Prod5": 0.3,
                             "Prod6": 0.2,
                             "Prod7": 0.3,},
           
              "vBohrer":{  "Prod1": 0.1,
                             "Prod2": 0.2,
                             "Prod4": 0.3,
                             "Prod6": 0.6,},
           
              "hBohrer":{  "Prod1": 0.2,
                             "Prod3": 0.8,
                             "Prod7": 0.6,},
           
              "Fräse":{      "Prod1": 0.05,
                             "Prod2": 0.03,
                             "Prod4": 0.07,
                             "Prod5": 0.1,
                             "Prod7": 0.08,},
           
              "Hobel":{      "Prod3": 0.01,
                             "Prod5": 0.05,
                             "Prod7": 0.05,},}

In [6]:
down = {("Jan","Schleifer"): 1,
        ("Feb","hBohrer"): 2,
        ("Mar","Fräse"): 1,
        ("Apr","vBohrer"): 1,
        ("May","Schleifer"): 1,
        ("May","vBohrer"): 1,
        ("Jun","Hobel"): 1,
        ("Jun","hBohrer"): 1,}

In [7]:
limit_dict = { "Jan":{  "Prod1": 500,
                        "Prod2": 1000,
                        "Prod3": 300,
                        "Prod4": 300,
                        "Prod5": 800,
                        "Prod6": 200,
                        "Prod7": 100,},
         
              "Feb":{   "Prod1": 600,
                        "Prod2": 500,
                        "Prod3": 200,
                        "Prod4": 0,
                        "Prod5": 400,
                        "Prod6": 300,
                        "Prod7": 150,},

                "Mar":{ "Prod1": 300,
                        "Prod2": 600,
                        "Prod3": 0,
                        "Prod4": 0,
                        "Prod5": 500,
                        "Prod6": 400,
                        "Prod7": 400,},

                "Apr":{ "Prod1": 200,
                        "Prod2": 300,
                        "Prod3": 400,
                        "Prod4": 500,
                        "Prod5": 200,
                        "Prod6": 0,
                        "Prod7": 100,},

                "May":{ "Prod1": 0,
                        "Prod2": 100,
                        "Prod3": 500,
                        "Prod4": 100,
                        "Prod5": 1000,
                        "Prod6": 300,
                        "Prod7": 0,},

                "Jun":{ "Prod1": 500,
                        "Prod2": 500,
                        "Prod3": 100,
                        "Prod4": 300,
                        "Prod5": 1100,
                        "Prod6": 500,
                        "Prod7": 60,},
    
}

limit = { (month , product) : limit_dict[month][product] for month in  months for product in products }

In [8]:
storeCost = 0.5
storeCapacity = 100
endStock = 50
hoursPerMonth = 2*8*24

In [9]:
model = Model('Factory')

Academic license - for non-commercial use only - expires 2021-07-26
Using license file C:\Users\tomwa\gurobi.lic


In [10]:
manu = model.addVars(months, products, name = "manu")
held = model.addVars(months, products, name = "held", ub = storeCapacity)
sell = model.addVars(months, products, name = "sell", ub = limit)

In [11]:
model.addConstrs((manu[months[0], product] == sell[months[0], product] 
                  + held[months[0], product] for product in products), name="balance")
   
model.addConstrs((held[months[month_index-1], product] + 
                 manu[month, product] == sell[month, product] + held[month, product] 
                 for product in products for month_index, month in enumerate(months) 
                 if month != months[0]), name="balance");

In [12]:
model.addConstrs((held[months[-1], product] == endStock for product in products),  name="End_Balance");

In [13]:
model.addConstrs((quicksum(time_table[machine][product] * manu[month, product] 
    for product in time_table[machine]) <= hoursPerMonth * (qMachine[machine] - 
    down[month, machine]) for machine in machines for month in months 
    if (month, machine) in down), name = "Capacity")

model.addConstrs((quicksum(time_table[machine][product] * manu[month, product] 
    for product in time_table[machine]) <= hoursPerMonth * qMachine[machine] 
    for machine in machines for month in months 
    if (month, machine) not in down), name = "Capacity");

In [14]:
obj = quicksum(
    profit[product] * sell[month, product] - storeCost * held[month, product]
    for month in months
    for product in products
)

model.setObjective(obj, GRB.MAXIMIZE)

In [15]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 79 rows, 126 columns and 288 nonzeros
Model fingerprint: 0x02693597
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [5e+01, 2e+03]
Presolve removed 74 rows and 109 columns
Presolve time: 0.01s
Presolved: 5 rows, 17 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2466500e+05   3.640000e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 2
       4    9.3758036e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  9.375803571e+04


In [16]:
model.printAttr('X')


    Variable            X 
-------------------------
manu[Jan,Prod1]          500 
manu[Jan,Prod2]      845.714 
manu[Jan,Prod3]        382.5 
manu[Jan,Prod4]          300 
manu[Jan,Prod5]          800 
manu[Jan,Prod6]          200 
manu[Jan,Prod7]          100 
manu[Feb,Prod1]          700 
manu[Feb,Prod2]          600 
manu[Feb,Prod3]        117.5 
manu[Feb,Prod5]          500 
manu[Feb,Prod6]          300 
manu[Feb,Prod7]          250 
manu[Mar,Prod6]          400 
manu[Apr,Prod1]          200 
manu[Apr,Prod2]          300 
manu[Apr,Prod3]          400 
manu[Apr,Prod4]          500 
manu[Apr,Prod5]          200 
manu[Apr,Prod7]          100 
manu[May,Prod2]          100 
manu[May,Prod3]          600 
manu[May,Prod4]          100 
manu[May,Prod5]         1100 
manu[May,Prod6]          300 
manu[May,Prod7]          100 
manu[Jun,Prod1]          550 
manu[Jun,Prod2]          550 
manu[Jun,Prod4]          350 
manu[Jun,Prod6]          550 
held[Jan,Prod3]         82.5 
held[Feb,Prod1] 

In [17]:
output = "<h1>Production plan</h1><table><tr><td></td><td><b>Manufacture</b></td><td><b>Sell</b></td><td><b>Hold</b></td></tr>"

for month in months:

    output += "<tr><td><b>{}</b></td><td style='text-align: right'>".format(month)
    
    # Manufacture
    for product in products:
        if manu[month, product].X > 0:
            output += "<b>{:.1f}</b> units of <b>{}</b><br/>".format(manu[month, product].X, product)
      
    # Sell
    output += "</td><td style='text-align: right'>"
    for product in products:
        if sell[month, product].X > 0:
            output += "<b>{:.1f}</b> units of <b>{}</b><br/>".format(sell[month, product].X, product)
            
    # Hold
    output += "</td><td style='text-align: right'>"
    for product in products:
        if held[month, product].X > 0:
            output += "<b>{:.1f}</b> units of <b>{}</b><br/>".format(held[month, product].X, product)
            
    output += "</td></tr>"
    
output += "</table>"

from IPython.display import HTML, display
display(HTML(output))

,Manufacture,Sell,Hold
Jan,500.0 units of Prod1845.7 units of Prod2382.5 units of Prod3300.0 units of Prod4800.0 units of Prod5200.0 units of Prod6100.0 units of Prod7,500.0 units of Prod1845.7 units of Prod2300.0 units of Prod3300.0 units of Prod4800.0 units of Prod5200.0 units of Prod6100.0 units of Prod7,82.5 units of Prod3
Feb,700.0 units of Prod1600.0 units of Prod2117.5 units of Prod3500.0 units of Prod5300.0 units of Prod6250.0 units of Prod7,600.0 units of Prod1500.0 units of Prod2200.0 units of Prod3400.0 units of Prod5300.0 units of Prod6150.0 units of Prod7,100.0 units of Prod1100.0 units of Prod2100.0 units of Prod5100.0 units of Prod7
Mar,400.0 units of Prod6,100.0 units of Prod1100.0 units of Prod2100.0 units of Prod5400.0 units of Prod6100.0 units of Prod7,
Apr,200.0 units of Prod1300.0 units of Prod2400.0 units of Prod3500.0 units of Prod4200.0 units of Prod5100.0 units of Prod7,200.0 units of Prod1300.0 units of Prod2400.0 units of Prod3500.0 units of Prod4200.0 units of Prod5100.0 units of Prod7,
May,100.0 units of Prod2600.0 units of Prod3100.0 units of Prod41100.0 units of Prod5300.0 units of Prod6100.0 units of Prod7,100.0 units of Prod2500.0 units of Prod3100.0 units of Prod41000.0 units of Prod5300.0 units of Prod6,100.0 units of Prod3100.0 units of Prod5100.0 units of Prod7
Jun,550.0 units of Prod1550.0 units of Prod2350.0 units of Prod4550.0 units of Prod6,500.0 units of Prod1500.0 units of Prod250.0 units of Prod3300.0 units of Prod450.0 units of Prod5500.0 units of Prod650.0 units of Prod7,50.0 units of Prod150.0 units of Prod250.0 units of Prod350.0 units of Prod450.0 units of Prod550.0 units of Prod650.0 units of Prod7
